In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [3]:

# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
desharnais=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\desharnais_dataset.csv",header=None,index_col=0)
columns_desharnais=['TeamExp','ManagerExp','YearEnd','Length','Effort','Transactions','Entities','PointsAdjust','Envergure','PointsNonAjust','Language']
desharnais.set_axis(columns_desharnais,axis='columns',inplace=True)
desharnais.rename_axis("Features", axis=1,inplace=True)
desharnais.rename_axis("Projects", axis=0,inplace=True)
dataset = desharnais

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Step 2: Preprocess Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Feature Selection using Harmony Search
def objective_function(features):
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    X_selected = X_scaled[:,features]
    error_list=[]
    
    for train_index, val_index in kf.split(X_selected):
        X_train, X_test = X_selected[train_index], X_selected[val_index]
        y_train, y_test = y[train_index], y[val_index]
        svr = SVR(kernel='linear')
        svr.fit(X_train, y_train)
        y_pred = svr.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        error_list.append(mae)
    return 1 / (1 + np.mean(error_list))
#====================================================================================================================

pbounds_HS = {
    'hms' : (10,30),
    'iterations': (30,100),
    'harmony_rate' : (0.1,0.9)
}

# Harmony Search Parameters
def harmony_search(hms,iterations, harmony_rate):
    num_features = X.shape[1]
    hms = int(hms)  # Harmony memory size
    iterations = int(iterations)  # Number of iterations

    # Initialize harmony memory
    harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

    # Harmony Search Algorithm
    for _ in range(iterations):
        rnd_choice=np.random.choice(hms)
        new_harmony = np.copy(harmony_memory[rnd_choice])
        for i in range(num_features):
            if np.random.rand() < harmony_rate:  # Adjust this probability based on your problem
                new_harmony[i] = 1 - new_harmony[i]
        current_obj = objective_function(harmony_memory[rnd_choice])
        new_obj = objective_function(new_harmony)
        if new_obj > current_obj:
            harmony_memory[rnd_choice] = new_harmony
    return max([objective_function(i) for i in harmony_memory])

optimizer_HS = BayesianOptimization(
        f=harmony_search,
        pbounds=pbounds_HS,
        random_state=42,
        verbose=1,
    )

optimizer_HS.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.
# Print the best hyperparameters found.
best_params_HS = optimizer_HS.max['params']

num_features = X_scaled.shape[1]
hms = int(best_params_HS['hms'])  # Harmony memory size
iterations = int(best_params_HS['iterations'])  # Number of iterations
harmony_rate= best_params_HS['harmony_rate']

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < harmony_rate:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj > current_obj:
        harmony_memory[rnd_choice] = new_harmony
    
#======================================================================================================================

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
pbounds = {'C': (0.1, 10),
           'epsilon': (0.01, 1),
           'gamma': (0.01, 100),
           'kernel': (0, 3)  # Use indices for categorical parameter
}
kernel_mapping = {0: 'linear', 1: 'rbf', 2: 'poly', 3: 'sigmoid'}

def svr_model(kernel,C, epsilon, gamma):
    model = SVR( kernel = kernel_mapping[int(kernel)], C=C, epsilon=epsilon, gamma=gamma)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.
    
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=svr_model,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )


    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
    print("Best Hyperparameters:")
    print(best_params)

    best_C = best_params['C']
    best_epsilon = best_params['epsilon']
    best_gamma = best_params['gamma']
    best_kernel= kernel_mapping[int(best_params['kernel'])]

    # Train the final SVR model with the best parameters
    final_model = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma,kernel= best_kernel)
    final_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = final_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
    
    
print(f"Mean Absolute Error: {np.mean(mean_MAE)}") 
print(f"Mean Magnitude of Relative Error (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE): {np.mean(mean_RMSE)}")


|   iter    |  target   | harmon... |    hms    | iterat... |
-------------------------------------------------------------
[1 1 1 0 0 1 1 0 1 0]
|   iter    |  target   |     C     |  epsilon  |   gamma   |  kernel   |
-------------------------------------------------------------------------
| 1         | -3.696e+0 | 3.808     | 0.9512    | 73.2      | 1.796     |
| 2         | -3.748e+0 | 1.645     | 0.1644    | 5.818     | 2.599     |
| 3         | -3.747e+0 | 6.051     | 0.711     | 2.068     | 2.91      |
| 4         | -3.693e+0 | 8.341     | 0.2202    | 18.19     | 0.5502    |
| 5         | -3.688e+0 | 3.112     | 0.5295    | 43.2      | 0.8737    |
| 6         | -3.695e+0 | 6.157     | 0.1481    | 29.22     | 1.099     |
| 7         | -3.696e+0 | 4.615     | 0.7873    | 19.98     | 1.543     |
| 8         | -3.691e+0 | 5.965     | 0.05599   | 60.76     | 0.5116    |
| 9         | -3.834e+0 | 0.744     | 0.9494    | 96.56     | 2.425     |
| 10        | -3.696e+0 | 3.116     | 0.

| 24        | -6.353e+0 | 9.529     | 0.7326    | 40.67     | 2.22      |
| 25        | -5.811e+0 | 0.1       | 0.01      | 30.81     | 3.0       |
| 26        | -5.789e+0 | 1.413     | 0.01      | 80.13     | 0.0       |
| 27        | -5.804e+0 | 0.4955    | 0.1094    | 24.82     | 0.4263    |
| 28        | -5.802e+0 | 7.537     | 1.0       | 11.15     | 3.0       |
| 29        | -5.788e+0 | 1.559     | 0.01      | 13.47     | 0.0       |
| 30        | -5.808e+0 | 0.2526    | 0.8999    | 86.73     | 0.2542    |
| 31        | -5.802e+0 | 10.0      | 1.0       | 33.41     | 3.0       |
| 32        | -5.733e+0 | 9.924     | 0.05952   | 6.115     | 0.2102    |
| 33        | -5.81e+03 | 0.1       | 1.0       | 0.01      | 0.0       |
| 34        | -5.872e+0 | 9.98      | 0.01702   | 58.09     | 2.355     |
| 35        | -5.805e+0 | 4.831     | 1.0       | 83.8      | 3.0       |
| 36        | -5.732e+0 | 10.0      | 0.01      | 0.01      | 0.0       |
| 37        | -5.811e+0 | 0.1       | 

| 9         | -4.515e+0 | 0.744     | 0.9494    | 96.56     | 2.425     |
| 10        | -4.648e+0 | 3.116     | 0.1067    | 68.43     | 1.32      |
| 11        | -4.615e+0 | 10.0      | 0.01      | 100.0     | 0.0       |
| 12        | -4.544e+0 | 0.1236    | 0.3701    | 90.69     | 2.069     |
| 13        | -4.644e+0 | 0.1       | 0.01      | 0.01      | 0.0       |
| 14        | -4.63e+03 | 8.245     | 0.01      | 7.645     | 3.0       |
| 15        | -4.644e+0 | 0.1       | 0.01      | 94.74     | 0.0       |
| 16        | -4.585e+0 | 9.389     | 0.7093    | 45.02     | 2.995     |
| 17        | -4.645e+0 | 7.774     | 0.8833    | 22.72     | 1.117     |
| 18        | -4.544e+0 | 2.016     | 0.625     | 24.83     | 2.303     |
| 19        | -4.649e+0 | 0.1624    | 0.4647    | 97.67     | 1.58      |
| 20        | -4.62e+03 | 7.452     | 0.5298    | 30.04     | 0.05903   |
| 21        | -4.644e+0 | 8.882     | 0.5357    | 25.7      | 1.594     |
| 22        | -4.645e+0 | 2.141     | 

| 33        | -5.157e+0 | 10.0      | 0.01      | 76.6      | 0.0       |
| 34        | -5.172e+0 | 8.86      | 0.6858    | 56.39     | 2.95      |
| 35        | -5.157e+0 | 10.0      | 0.01      | 52.18     | 0.0       |
| 36        | -5.235e+0 | 0.1       | 1.0       | 45.17     | 0.0       |
| 37        | -5.161e+0 | 6.947     | 0.07053   | 79.83     | 0.0961    |
| 38        | -5.157e+0 | 10.0      | 0.01      | 12.5      | 0.0       |
| 39        | -5.157e+0 | 10.0      | 1.0       | 21.64     | 0.0       |
| 40        | -5.167e+0 | 6.069     | 0.871     | 50.46     | 0.4911    |
Best Hyperparameters:
{'C': 10.0, 'epsilon': 1.0, 'gamma': 21.64450102334453, 'kernel': 0.0}
Mean Absolute Error: 2955.3466232136902
Mean Magnitude of Relative Error (MMRE): 0.91
Root Mean Squared Error (RMSE): 4408.068288251898
